# Pickle 模块测试套件 - 稳定性与正确性分析


In [1]:
import pickle
import hashlib
import sys
import platform
import datetime
import os


In [2]:
def get_pickle_hash(obj):
    try:
        data = pickle.dumps(obj)
        return hashlib.sha256(data).hexdigest()
    except Exception as e:
        return f"ERROR: {e}"


## 功能性测试（Functional Testing）：基础类型的序列化


In [3]:
test_data = {
    "整数": 42,
    "浮点数": 3.14159,
    "布尔值": True,
    "字符串": "ChatGPT",
    "列表": [1, 2, 3],
    "字典": {"a": 1, "b": 2},
    "元组": (4, 5, 6),
    "集合": {7, 8, 9},
    "嵌套结构": {"x": [1, {"y": (3, 4)}]},
    "空值": None
}

results = {}
for name, obj in test_data.items():
    try:
        serialized = pickle.dumps(obj)
        deserialized = pickle.loads(serialized)
        passed = obj == deserialized
    except Exception as e:
        passed = False
        deserialized = str(e)
    results[name] = {
        "原始对象": obj,
        "反序列化结果": deserialized,
        "是否一致": passed
    }

results


{'整数': {'原始对象': 42, '反序列化结果': 42, '是否一致': True},
 '浮点数': {'原始对象': 3.14159, '反序列化结果': 3.14159, '是否一致': True},
 '布尔值': {'原始对象': True, '反序列化结果': True, '是否一致': True},
 '字符串': {'原始对象': 'ChatGPT', '反序列化结果': 'ChatGPT', '是否一致': True},
 '列表': {'原始对象': [1, 2, 3], '反序列化结果': [1, 2, 3], '是否一致': True},
 '字典': {'原始对象': {'a': 1, 'b': 2}, '反序列化结果': {'a': 1, 'b': 2}, '是否一致': True},
 '元组': {'原始对象': (4, 5, 6), '反序列化结果': (4, 5, 6), '是否一致': True},
 '集合': {'原始对象': {7, 8, 9}, '反序列化结果': {7, 8, 9}, '是否一致': True},
 '嵌套结构': {'原始对象': {'x': [1, {'y': (3, 4)}]},
  '反序列化结果': {'x': [1, {'y': (3, 4)}]},
  '是否一致': True},
 '空值': {'原始对象': None, '反序列化结果': None, '是否一致': True}}

## 稳定性测试：相同对象的 Pickle 哈希值是否一致


In [4]:
sample_object = {"test": [1, 2, 3], "time": datetime.datetime(2020, 1, 1)}

hash1 = get_pickle_hash(sample_object)
hash2 = get_pickle_hash(sample_object)
hash3 = get_pickle_hash(sample_object)

print("第一次哈希:", hash1)
print("第二次哈希:", hash2)
print("第三次哈希:", hash3)
print("一致性:", hash1 == hash2 == hash3)


第一次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
第二次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
第三次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
一致性: True


## 模糊测试示例：随机生成复杂对象并检查序列化稳定性


In [5]:
import random

def generate_random_nested_structure(depth=3):
    if depth == 0:
        return random.choice([42, 3.14, True, "text", None])
    structure_type = random.choice(["list", "dict", "tuple"])
    if structure_type == "list":
        return [generate_random_nested_structure(depth - 1) for _ in range(2)]
    elif structure_type == "dict":
        return {str(i): generate_random_nested_structure(depth - 1) for i in range(2)}
    else:
        return tuple(generate_random_nested_structure(depth - 1) for _ in range(2))

fuzz_object = generate_random_nested_structure()
hash_before = get_pickle_hash(fuzz_object)
hash_after = get_pickle_hash(fuzz_object)

print("生成的结构:", fuzz_object)
print("哈希一致性:", hash_before == hash_after)


生成的结构: ({'0': [None, 42], '1': {'0': 3.14, '1': True}}, [{'0': 42, '1': 3.14}, ('text', 3.14)])
哈希一致性: True
